# Explicabilidad en Inteligencia Artificial

## 1. Introducción

Debido al incremento exponencial del desarrollo y la integración de sistemas inteligentes en actividades de diversa naturaleza y frecuencia, la comprensión de su funcionamiento resulta fundamental por diversos motivos. 
En primer lugar si un modelo pretende **desempeñar un papel fundamental como una herramienta para ayudar a la toma de decisiones**, su modo de razonamiento debe ser fácilmente entendible por un ser humano tradicional. Así se puede analizar su **comportamiento** para observar rasgos indeseados que poder corregir, asegurar que sus pilares se encuentran alineados con los objetivos del problema que aborda, que cumple una determinada **ética** establecida en torno a las soluciones que propone, además de medidas de seguridad críticas con las que controlar la **calidad** de las mismas. 

De este modo en este notebook se pretende aplicar distintas técnicas de explicabilidad con la que comprender más detalladamente la conducta característica de los clasificadores elaborados para la detección de textos sexistas.

### 1.1. Tipos de explicabilidad

* **Modelos agnósticos locales**. Esta categoría de técnicas se pueden emplear sobre cualquier tipo de modelo de caja negra aunque únicamente se centran en una sola muestra.

## 2. Estructura del notebook

1. Introducción a la Explicabilidad en Inteligencia Artificial
2. Estructura del notebook
3. Instalación y carga de librerías
4. Lectura y carga de datos procesados
5. Técnicas de explicabilidad
6. Conclusiones

## 3. Instalación y carga de librerías

Este apartado tiene como único propósito cargar las librerías y dependencias necesarias para la ejecución de este notebook, así como las funciones propiamente desarrolladas. Previo a ello deberán ser instaladas bien ejecutando el script *setup.sh* mediante el comando `bash setup.sh` con permisos de ejecución en distribuciones Linux, o bien ejecutando el compando `pip install -r requirements.txt`.

In [1]:
%%capture
import sys
sys.path.append('../scripts')

# Import encoding functions
from encoding import *

# pandas: to read the datasets
import pandas as pd

# numpy: to work with predicted probabilities
import numpy as np

# skelearn: to encode documents
from sklearn.feature_extraction.text import CountVectorizer

# pickle: to load Logistic Regression models
import pickle 

# keras: to load pre-trained models
from keras.utils import pad_sequences
from keras.models import load_model
from keras.preprocessing.text import Tokenizer

# lime: to create local explanations
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer

# random: to choose random samples
from random import randrange

2023-04-17 13:55:05.161588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 13:55:05.541508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-17 13:55:05.541548: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-17 13:55:06.861229: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## 4. Lectura y carga de datos procesados

En esta sección se pretende leer y cargar los datos de entrenamiento y test ya procesados a los que se les ha aplicado las siguientes técnicas de tratamiento de documentos:

  - Elimina URLs.
  - Elimina usuarios mencionados.
  - Elimina caracteres especiales, no alfabéticos y signos de puntuación.
  - Convierte todos los caracteres en minúsculas.

Tal y como se puede comprobar en los siguientes resultados las dimensiones de sendos conjuntos de datos se detallan a continuación:

* Conjunto de entrenamiento: **6.865 muestras**.
* Conjunto de validación: **4.296 muestras**.

In [2]:
# Read already processed EXIST datasets
train_df = pd.read_csv('../data/proc_EXIST2021_train.csv')
test_df = pd.read_csv('../data/proc_EXIST2021_test.csv')

# Show the dimensions of the datasets
print('Train dataset dimensions:', train_df.shape)
print('Test dataset dimensions:', test_df.shape)

Train dataset dimensions: (6865, 6)
Test dataset dimensions: (4296, 6)


## 5. Técnicas de explicabilidad

### 5.1. LIME

Se trata de un procedimiento que genera **explicaciones locales a una única muestra dentro de una determinada clase** cuyo objetivo consiste en determinar la relevancia de cada característica (palabra) proporcionada para su predicción. Para ello genera un **modelo lineal** que se entrena en base a perturbaciones generadas a partir de un ejemplo seleccionado y luego es evaluado dependiendo de la similitud entre las predicciones producidas y la real asociada a la entrada suministrada. Esta tarea es posible mediante el uso de la librería *lime* de Python a la que se le debe proporcionar una muestra de ejemplo junto con una función que permite obtener la predicción de las perturbaciones creadas.

Puesto que se trata de un método local a un ejemplo, se propone su aplicación en los conjuntos de **falsos positivos y negativos** analizados en notebooks anteriores con la idea de comprobar cuáles son los términos destacables que han liderado hacia una clasificación errónea. Como los volumenes de muestras son bastante considerables, se pretende seleccionar **diez ejemplos aleatoriamente de cada intervalo de confianza** para además detectar la existencia de características diferentes en función de la seguridad demostrada por parte del modelo al identificar cada dato de test.

#### 5.1.1. Modelo inglés de Regresión Logística

Comenzamos ejecutando este método sobre el mejor clasificador inglés de Regresión Logística encontrado cuya codificación se realizó transformando los textos en bolsas de palabras. A continuación se observa que este modelo se caracteriza por tener 213 falsos positivos y 438 falsos negativos, unas cifras de errores demasiado elevadas para considerarlo como una solución de calidad.

In [3]:
# Load a Logistic Regression model specialized in English
en_lr_model = pickle.load(open('../models/multileng_lr_models/en_bag_words_model.sav', 'rb'))

# Filter the train documents by language
en_train_df = train_df[train_df['language'] == 'en']
en_train_texts = list(en_train_df['clean_text'].values)

# Filter the test documents by language
en_test_df = test_df[test_df['language'] == 'en']
en_test_texts = list(en_test_df['clean_text'].values)

# Convert train and test documents to bag of words
en_train_bag_words, en_test_bag_words = to_bag_of_words(
    train_docs=en_train_texts, 
    test_docs=en_test_texts)

# Predict over the test dataset
en_test_df['task1_pred_classes'] = en_lr_model.predict(X=en_test_bag_words)
en_test_df['task1_pred_probs'] = [max(prob) for prob in en_lr_model.predict_proba(X=en_test_bag_words)]

# Get the indexes of the false negatives and positives
en_fn_df = en_test_df[(en_test_df['task1'] == 1) & (en_test_df['task1_pred_classes'] == 0)]
en_fp_df = en_test_df[(en_test_df['task1'] == 0) & (en_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {en_fn_df.shape[0]}')
print(f'No. of false positives: {en_fp_df.shape[0]}\n')

No. of false negatives: 438
No. of false positives: 213



/tmp/ipykernel_8388/2998631689.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_classes'] = en_lr_model.predict(X=en_test_bag_words)
/tmp/ipykernel_8388/2998631689.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_probs'] = [max(prob) for prob in en_lr_model.predict_proba(X=en_test_bag_words)]


Las conclusiones extraídas de la generación de explicaciones locales a un subconjunto de **falsos negativos** se presentan a continuación:

* Los falsos negativos pertenecientes al más elevado intervalo de confianza otorga un grado de **no-sexismo a palabras sin significado** como pronombres, determinantes, preposiciones y verbos auxiliares, que pueden contrarrestar el efecto de terminología sexista. 

* Aquellos cometidos bajo un rango de confianza alto demuestran un notable grado de **ironía** con terminología positiva aunque con significados profundamente sexistas. Este fenómeno ya se intuía en el análisis exploratorio de datos que se realizó al comienzo de este proyecto.

* Finalmente los falsos negativos que se encuentran dentro del intervalo moderado de confianza están caracterizados por su alta composición de **hashtags y errores ortográficos** que dificultan su análisis e identificación automáticos. No obstante, pese a haber tratado de detectar y corregir los términos gramaticalmente erróneos empleando diversas librerías, ninguna de ellas ha podido conseguir la mejora de la capacidad de predicción de los modelos de Regresión Logística.

In [ ]:
# Create a LIME explainer object
lime_explainer = LimeTextExplainer(class_names={
    0: 'non-sexist',
    1: 'sexist'
})

for pair in [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]:
    # Filter the false negatives by confidence interval
    en_fn_subdf = en_fn_df[(en_fn_df['task1_pred_probs'] >= pair[0]) & \
        (en_fn_df['task1_pred_probs'] < pair[1])]
    en_fn_subdf_texts = list(en_fn_subdf['clean_text'].values)

    # Choose 10 distinct samples randomly
    chosen_fn_texts = []
    for iter in range(0, 10):
        fn_index = randrange(len(en_fn_subdf_texts))
        while (fn_index in chosen_fn_texts):
            fn_index = randrange(len(en_fn_subdf_texts))

        chosen_fn_texts.append(fn_index)

    # Create explanations for each sample
    def predict_en_lrm_model(sample: str):

        # Train a CountVectorizer object based on train documents
        bg_vectorizer = CountVectorizer()
        bg_vectorizer.fit(raw_documents=en_train_texts)

        # Encode the provided test sample 
        encoded_sample = bg_vectorizer.transform(raw_documents=sample).toarray()
        
        # Predict the probabilities for both classes
        pred_prob = en_lr_model.predict_proba(X=encoded_sample)
        return np.array(pred_prob)

    for indx in chosen_fn_texts:
        # Create a single explanation for a single test sample
        lime_explanation = lime_explainer.explain_instance(
            en_fn_subdf_texts[indx], 
            predict_en_lrm_model).show_in_notebook(text=True)

Replicando el mismo procedimiento anterior aunque aplicado a los **falsos positivos** se han podido descubrir las siguientes pesquisas:

* Los falsos positivos de los dos intervalos superiores de confianza **contienen noticias y reivindaciones de usuarios contra los ataques y la violencia contra la mujer**. Como se detalló en el análisis exploratorio de datos, las temáticas del conjunto de test son bien distintas de las tratadas en el dataset de entrenamiento. El problema reside en que términos comunes como "mujeres", "feminismo", etc. son catalogados individualmente como sexistas y como consecuencia producen este comportamiento erróneo en el modelo.

* Los falsos positivos situados en el intervalo moderado de confianza destacan por tratar **otros tópicos** en los que aparecen también palabras de género como "mujeres", "marido", entre otras que pese a ser clasificadas como sexistas su significado no es tal. De nuevo, este hecho fue descubierto durante el análisis exploratorio de datos inicial en el que ya se aventuraba las discrepancias de contenido entre ambos datasets.

#### 5.1.2. Modelo español de Regresión Logística

A continuación se repite el estudio previo pero haciendo uso del mejor modelo español de Regresión Logística con sus correspondientes documentos en dicho idioma. En los siguientes resultados se puede apreciar que existen 462 documentos sexistas no detectados y 199 textos no sexistas erróneamente clasificados.

In [6]:
# Load a Logistic Regression model specialized in Spanish
es_lr_model = pickle.load(open('../models/multileng_lr_models/es_bag_words_model.sav', 'rb'))

# Filter the train documents by language
es_train_df = train_df[train_df['language'] == 'es']
es_train_texts = list(es_train_df['clean_text'].values)

# Filter the test documents by language
es_test_df = test_df[test_df['language'] == 'es']
es_test_texts = list(es_test_df['clean_text'].values)

# Convert train and test documents to bag of words
es_train_bag_words, es_test_bag_words = to_bag_of_words(
    train_docs=es_train_texts, 
    test_docs=es_test_texts)

# Predict over the test dataset
es_test_df['task1_pred_classes'] = es_lr_model.predict(X=es_test_bag_words)
es_test_df['task1_pred_probs'] = [max(prob) for prob in es_lr_model.predict_proba(X=es_test_bag_words)]

# Get the indexes of the false negatives and positives
es_fn_df = es_test_df[(es_test_df['task1'] == 1) & (es_test_df['task1_pred_classes'] == 0)]
es_fp_df = es_test_df[(es_test_df['task1'] == 0) & (es_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {es_fn_df.shape[0]}')
print(f'No. of false positives: {es_fp_df.shape[0]}\n')

No. of false negatives: 462
No. of false positives: 199



/tmp/ipykernel_8388/1758952928.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_classes'] = es_lr_model.predict(X=es_test_bag_words)
/tmp/ipykernel_8388/1758952928.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_probs'] = [max(prob) for prob in es_lr_model.predict_proba(X=es_test_bag_words)]


Las deducciones descubiertas tras la visualización de diez muestras positivas clasificadas erróneamente seleccionadas aleatoriamente por cada intervalo de confianza, como se ha efectuado con el clasificador inglés, han sido idénticamente similares a las observadas previamente.

Por otro lado en relación a los falsos positivos, es decir ejemplos negativos identificados como positivos, además de compartir las características descubiertas anteriormente también cabe destacar la **sexualización por parte del modelo de prendas femeninas**, como la falda, además de la no comprensión de terminología latinoamericana puesto que no se les asigna ninguna clase o valor por lo que se consideran neutros.

#### 5.1.3. Modelo LSTM inglés

En esta sección se cambia de arquitectura para generar exlicaciones locales utilizando el mejor modelo LSTM específico de documentos ingleses compuesto por capas bidireccionales. En los siguientes resultados se puede observar la existencia de más de 398 falsos negativos y 246 falsos positivos. A continuación se procede a muestrear diez ejemplos de cada conjunto por intervalo de confianza para analiar sus resultados, tal y como se realizó en los apartados previos.

In [4]:
def encode_data_as_matrixes(train_texts:list, test_texts: list, max_n_words: int, sequence_len: int):
    '''
    Function that creates and trains a Keras tokenizer based on the provided
    train documents to encode the train and test data as numeric matrixes.

    Parameters
    ----------
    train_texts : list
        A list of strings with the train documents to encode.
    test_texts : list
        A list of strings with the test documents to encode.
    max_n_words : int
        Maximum number of words to keep within the LSTM memory
        based on computing the word frequency.
    sequence_len : int
        Maximum lenght of all sequences.
    
    Returns
    -------
    A
    '''
    # Create a tokenizer based on train texts
    tokenizer = Tokenizer(num_words=max_n_words)
    tokenizer.fit_on_texts(train_texts)

    # Transform each text into a numeric sequence
    train_sequences = tokenizer.texts_to_sequences(train_texts)

    # Transform each numeric sequence into a 2D vector
    train_matrix = pad_sequences(
        sequences=train_sequences, 
        maxlen=sequence_len)

    # Tokenize the test documents using the prior trained tokenizer
    test_sequences = tokenizer.texts_to_sequences(test_texts)

    # Transform each numeric sequence into a 2D vector
    test_matrix = pad_sequences(
        sequences=test_sequences,
        maxlen=sequence_len)
    
    return tokenizer, train_matrix, test_matrix

In [6]:
# Number of words to retain in the LSTM memory
MAX_N_WORDS = 1000

# Max number of tokens per numeric sequence
SEQUENCE_MAX_LEN = 100

# Path to the multilanguage embedding file
MULTILAN_EMBEDDINGS_FILE = '../en_es_embeddings/glove.twitter.27B.100d.txt'

# Filter the train documents by language
en_train_df = train_df[train_df['language'] == 'en']
en_train_texts = list(en_train_df['clean_text'].values)

# Filter the test documents by language
en_test_df = test_df[test_df['language'] == 'en']
en_test_texts = list(en_test_df['clean_text'].values)

# Convert processed train and test documents into matrixes
en_tokenizer, en_train_matrix, en_test_matrix = encode_data_as_matrixes(
    train_texts=en_train_texts,
    test_texts=en_test_texts,
    max_n_words=MAX_N_WORDS,
    sequence_len=SEQUENCE_MAX_LEN
)

# Load a pretrained BiLSTM model trained over English texts
en_bilstm_model = load_model('../models/en_lstm_models/en_bilstm_model_2L_128N_32BS.h5')

# Predict over the test dataset
en_test_df['task1_pred_probs'] = (en_bilstm_model.predict(en_test_matrix))
en_test_df['task1_pred_classes'] = (en_bilstm_model.predict(en_test_matrix) >= 0.5).astype('int32')

# Get the indexes of the false negatives and positives
en_fn_df = en_test_df[(en_test_df['task1'] == 1) & (en_test_df['task1_pred_classes'] == 0)]
en_fp_df = en_test_df[(en_test_df['task1'] == 0) & (en_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {en_fn_df.shape[0]}')
print(f'No. of false positives: {en_fp_df.shape[0]}\n')

2023-04-17 13:20:55.146273: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-17 13:20:55.146531: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-17 13:20:55.146553: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lidiasm): /proc/driver/nvidia/version does not exist
2023-04-17 13:20:55.147093: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 2/69 [..............................] - ETA: 4s

/tmp/ipykernel_5433/1486523555.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_probs'] = (en_bilstm_model.predict(en_test_matrix))


69/69 [==============================] - 4s 64ms/step
No. of false negatives: 398
No. of false positives: 246



/tmp/ipykernel_5433/1486523555.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_classes'] = (en_bilstm_model.predict(en_test_matrix) >= 0.5).astype('int32')


Del código propuesto anteriormente únicamente se ha modificado la función que calcula las predicciones para las perturbaciones basadas en una muestra de test ya que es requisito indispensable obtener una probabilidad para cada clase aunque el método de Keras únicamente devuelve la más elevada. Tras inspeccionar los resultados de los **falsos negativos** se pueden concluir que se repiten los fenómenos analizados en los modelos anteriores y que de nuevo se han visualizado ciertos **términos que son claramente sexistas pero no han sido tratados como tal**, lo que ha contribuido a cometer clasificaciones erroneas. 

In [ ]:
# Create a LIME explainer object
lime_explainer = LimeTextExplainer(class_names={
    0: 'non-sexist',
    1: 'sexist'
})

for pair in [(0.4, 0.6), (0.2, 0.4), (-0.1, 0.2)]:
    # Filter the false negatives by confidence interval
    en_fn_subdf = en_fn_df[(en_fn_df['task1_pred_probs'] >= pair[0]) & \
        (en_fn_df['task1_pred_probs'] < pair[1])]
    en_fn_subdf_texts = list(en_fn_subdf['clean_text'].values)

    # Choose 10 distinct samples randomly
    chosen_fn_texts = []
    for iter in range(0, 10):
        fn_index = randrange(len(en_fn_subdf_texts))
        while (fn_index in chosen_fn_texts):
            fn_index = randrange(len(en_fn_subdf_texts))

        chosen_fn_texts.append(fn_index)

    # Create explanations for each sample
    def encode_predict_lstm_model(sample: str):

        # Tokenize the test documents using the prior trained tokenizer
        sample_sequence = en_tokenizer.texts_to_sequences(sample)

        # Transform each numeric sequence into a 2D vector
        encoded_sample = pad_sequences(
            sequences=sample_sequence,
            maxlen=SEQUENCE_MAX_LEN)
            
        # Predict the probability for the max class
        one_class_pred_probs = en_bilstm_model.predict(encoded_sample)

        # Calculate the probability for each class
        biclass_pred_probs = [np.array([1-record[0], record[0]]) for record in one_class_pred_probs]
        return np.array(biclass_pred_probs)
    

    for indx in chosen_fn_texts:
        # Create a single explanation for a single test sample
        lime_explanation = lime_explainer.explain_instance(
            en_fn_subdf_texts[indx], 
            predict_en_lrm_model).show_in_notebook(text=True)

En referencia a los **falsos positivos** existen una gran multitud de similitudes con respecto al análisis previo de muestras negativas erróneamente clasificadas, de nuevo he podido descubrir textos que aluden a otras temáticas y que han sido categorizados como positivos por contener palabras de género como "woman", "women", "men", etc. Sin embargo, un fenómeno muy destacable ha sido el hecho de encontrar documentos que contienen **negativas de términos sexistas** pero que han sido identificados como positivos puesto que el modelo no ha sido capaz de observar la expresión en su conjunto.

#### 5.1.4. Modelo LSTM español

A continuación se procede a replicar el procedimiento analítico anterior aunque sobre el mejor modelo LSTM encontrado para documentos en español. Tal y como se aprecia en los siguientes resultados, dispone de 392 falsos negativos y 210 falsos positivos.

In [5]:
# Number of words to retain in the LSTM memory
MAX_N_WORDS = 1000

# Max number of tokens per numeric sequence
SEQUENCE_MAX_LEN = 100

# Path to the multilanguage embedding file
MULTILAN_EMBEDDINGS_FILE = '../en_es_embeddings/glove.twitter.27B.100d.txt'

# Filter the train documents by language
es_train_df = train_df[train_df['language'] == 'es']
es_train_texts = list(es_train_df['clean_text'].values)

# Filter the test documents by language
es_test_df = test_df[test_df['language'] == 'es']
es_test_texts = list(es_test_df['clean_text'].values)

# Convert processed train and test documents into matrixes
es_tokenizer, es_train_matrix, es_test_matrix = encode_data_as_matrixes(
    train_texts=es_train_texts,
    test_texts=es_test_texts,
    max_n_words=MAX_N_WORDS,
    sequence_len=SEQUENCE_MAX_LEN
)

# Load a pretrained BiLSTM model trained over English texts
es_lstm_model = load_model('../models/es_lstm_models/es_lstm_3L_1286432N_16BS.h5')

# Predict over the test dataset
es_test_df['task1_pred_probs'] = (es_lstm_model.predict(es_test_matrix))
es_test_df['task1_pred_classes'] = (es_lstm_model.predict(es_test_matrix) >= 0.5).astype('int32')

# Get the indexes of the false negatives and positives
es_fn_df = es_test_df[(es_test_df['task1'] == 1) & (es_test_df['task1_pred_classes'] == 0)]
es_fp_df = es_test_df[(es_test_df['task1'] == 0) & (es_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {es_fn_df.shape[0]}')
print(f'No. of false positives: {es_fp_df.shape[0]}\n')

2023-04-17 13:55:30.692132: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-17 13:55:30.692380: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-17 13:55:30.692396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lidiasm): /proc/driver/nvidia/version does not exist
2023-04-17 13:55:30.692890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 3/67 [>.............................] - ETA: 2s

/tmp/ipykernel_4270/702147442.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_probs'] = (es_lstm_model.predict(es_test_matrix))


67/67 [==============================] - 2s 32ms/step
No. of false negatives: 392
No. of false positives: 210



/tmp/ipykernel_4270/702147442.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_classes'] = (es_lstm_model.predict(es_test_matrix) >= 0.5).astype('int32')


Los documentos submuestreados para una inspección visual y una explicabilidad local de los **falsos negativos** han apuntado hacia nuevos fenómenos explicativos del pésimo comportamiento del modelo. En primer lugar existen textos que no contienen terminología sexista pero sí que pertenecen a esta clase. Sin embargo, desde una perspectiva técnica y automática no contienen pistas que hagan pensar que se trata de muestras positivas, para ello sería necesario un **mayor nivel de entendimiento del lenguaje natural** por parte del modelo que se asimilase al del humano. Por otro lado también son ampliamente utilizados **conceptos informales y casuales**, que no se encuentran comunmente en los diccionarios oficiales de idiomas, y que por tanto no son reconocidos automáticamente provocando una pérdida de información vital.

No obstante en el estudio de los **falsos positivos** las pesquisas obtenidas son las ya conocidas hasta el momento con documentos con terminología propia de la temática considerada como sexista liderando los fallos cometidos, así como reivindicaciones con expresiones sexistas pero con un significado positivo.

## 6. Conclusiones generales

* En primer lugar cabe destacar el hecho de que LIME únicamente analiza términos individuales en lugar de conjuntos de palabras o expresiones que podría ser más útil ya que conceptos por sí solos pueden no ser sexistas a no ser que se encuentren acompañados de otros que le otorguen ese significado, como violencia de género.

* En los modelos de Regresión Logística y LSTM se ha podido descubrir que **terminología que debía ser neutra**, como pronombres, determinantes, preposiciones, se les ha asignado un valor no sexista que contrarresta, por su elevado volumen de aparación, a los conceptos que sí lo son provocando una multitud de muestras erróneamente clasificadas. Igualmente ocurre con **palabras comunes a este tópico**, como mujeres, marido, hombres, que son tratadas como sexistas de forma individual lo que conlleva también un considerable número de fallos.

* Otra muestra más del trabajo que queda por realizar para aumentar el nivel de comprensión del lenguaje natural por modelos automáticos reside en aquellos documentos que muestran un **elevado grado de ironía o contienen negaciones de expresiones sexistas** que son categorizados dentro de la clase positiva.

* Finalmente se ha descubierto que tanto los hashtags como los errores gramaticales producen una falta de información, severa en ciertos documentos, que provocan la pérdida de información liderando hacia una clasificación errónea de las muestras. Ambos fenómenos se encuentran más presentes en los ejemplos erróneamente clasificados bajo **intervalos moderados y bajos de confianza**, puesto que en estos casos el modelo apenas dispone de términos útiles en los que fundamentar su decisión y por ende se muestra más inseguro.